# Wprowadzenie 
Nasze wyzwanie jest z jednej strony proste, z drugiej strony dość ambitne. 

Jedno z klasycznych "Hello World" świata Big Data polega na zliczaniu wystąpienia słów. Dane wejściowe - plik tekstowy lub strumień tekstu. Dane wynikowe - liczba wystąpień każdego ze słów. Klasyka. 

My zrobimy to samo, jednak naszymi danymi wejściowymi będą... opowiadania Artura Conan Doyla (czyli standard), ale nie w plikach tekstowych, a w formacie PDF (i to już standard nie jest). 

Trudne? Nic bardziej mylnego. Python to mnogość bibliotek o niezliczonej funkcjonalności. 

Prosty przykład...

Pobierz nasze dane wejściowe

In [1]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import sys
import os

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

conf = SparkConf().setAppName("Spark - RDD par")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName("Spark - RDD par").getOrCreate()

In [2]:
import requests
r = requests.get("https://jankiewicz.pl/bigdata/bigdata-sp/cano-pdf.zip", allow_redirects=True)
open('cano-pdf.zip', 'wb').write(r.content)

7830123

Rozpakuj nasz plik

In [3]:
%%sh
unzip cano-pdf.zip

Couldn't find program: 'sh'


Sprawdź czy mamy zainstalowany potrzebny moduł

# PyPDF2

In [ ]:
%%sh
pip freeze | grep PyPDF2

In [4]:
import PyPDF2 
    
# Utwórz obiekt odnoszący się do przykładowego pliku
pdfFileObj = open('cano-pdf/3gab.pdf', 'rb') 
    
# Utwórz obiekt PdfFileReader 
pdfReader = PyPDF2.PdfReader(pdfFileObj) 
    
# To wszystko 
# Zobacz ile ten plik ma stron 
print(len(pdfReader.pages))

10


In [5]:
# Pobierz pierwszą ze stron
pageObj = pdfReader.pages[0]
    
# Dokonaj esktrakcji tekstu, który się na niej znajduje 
print(pageObj.extract_text()) 

The Adventure of the Three Gables
Arthur Conan Doyle


In [6]:
# Nie zapomnij zamknąć nasz obiekt pliku
pdfFileObj.close() 

Proste prawda? 

No to do roboty. W pierwszej kolejności załadujmy dane tam, gdzie będą one mogły być wydajnie odczytywane przez wiele węzłów klastra

# Przygotowanie danych

In [ ]:
%%sh
hadoop fs -mkdir -p cano-pdf

In [ ]:
%%sh
hadoop fs -put -f cano-pdf/* cano-pdf/

In [ ]:
%%sh
hadoop fs -ls cano-pdf

Utwórzmy teraz nasz obiekt konteksu (o ile jeszcze nie istnieje)

# Utworzenie obiektu kontekstu

In [ ]:
# w przypadku korzystania z kernela Python
from pyspark import SparkContext, SparkConf

In [ ]:
# w przypadku korzystania z kernela Python
conf = SparkConf().setAppName("Spark - RDD - warsztaty").setMaster("yarn")
sc = SparkContext(conf=conf)

Do tej pory szło gładko. Teraz mamy mały problem. <br> 
W jaki sposób zaczytać nasze pliki? 

Nie są to pliki tekstowe, więc `textFile` prowadzający dane linia po linii do naszych dokumentów nie jest tu przydatny.<br>
Zaglądnij na https://spark.apache.org/docs/latest/rdd-programming-guide.html#external-datasets

Właściwie, żadna z metod nie jest tu odpowiednia. 

Zrobimy zatem tak, naszymi danymi wejściowymi nie będą pliki. Będą ich nazwy, a Spark na podstawie tych nazw będzie je odczytywał i ... 

# Przygotowanie metadanych wejściowych

In [7]:
sc

<SparkContext master=local[*] appName=Spark - RDD par>

In [ ]:
%%sh
hadoop fs -ls cano-pdf > files.txt

In [ ]:
%%sh
hadoop fs -copyFromLocal files.txt

In [11]:
rawFiles = sc.textFile("files.txt")

In [12]:
rawFiles.collect()

['3gab.pdf',
 '3gar.pdf',
 '3stu.pdf',
 'abbe.pdf',
 'bery.pdf',
 'blac.pdf',
 'blan.pdf',
 'blue.pdf',
 'bosc.pdf',
 'bruc.pdf',
 'card.pdf',
 'chas.pdf',
 'copp.pdf',
 'cree.pdf',
 'croo.pdf',
 'danc.pdf',
 'devi.pdf',
 'dyin.pdf',
 'empt.pdf',
 'engr.pdf',
 'fina.pdf',
 'five.pdf',
 'glor.pdf',
 'gold.pdf',
 'gree.pdf',
 'houn.pdf',
 'iden.pdf',
 'illu.pdf',
 'lady.pdf',
 'last.pdf',
 'lion.pdf',
 'maza.pdf',
 'miss.pdf',
 'musg.pdf',
 'nava.pdf',
 'nobl.pdf',
 'norw.pdf',
 'prio.pdf',
 'redc.pdf',
 'redh.pdf',
 'reig.pdf',
 'resi.pdf',
 'reti.pdf',
 'scan.pdf',
 'seco.pdf',
 'shos.pdf',
 'sign.pdf',
 'silv.pdf',
 'sixn.pdf',
 'soli.pdf',
 'spec.pdf',
 'stoc.pdf',
 'stud.pdf',
 'suss.pdf',
 'thor.pdf',
 'twis.pdf',
 'vall.pdf',
 'veil.pdf',
 'wist.pdf',
 'yell.pdf']

Jesteśmy zainteresowani tylko nazwami plików, a zatem...

In [15]:
import re
rawFiles

files.txt MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:0

In [24]:
fileNames = rawFiles.map(lambda filename: "cano-pdf/" + filename)

Nie chcemy aby całą ekstrakcję danych tekstowych z plików PDF wykonywał jeden węzeł. Sprawdźmy ile mamy partycji naszego RDD. 

Jeśli będzie ich zbyt mało, możemy zmienić ich liczbę za pomoca metody `repartition(liczba_partycji)`.

Przeanalizuj to ile zasobów ma nasz klaster, w szczególności zwróć uwagę na liczbę procesorów we wszystkich maszynach.

Stosując *regułę kciuka* ustaw liczbę partycji na taką, która jest równa liczbie procesorów. Wprowadź zmiany w powyższej linii, tak aby poniższa potwierdziła oczekiwaną liczbę partycji. 

In [17]:
fileNames.getNumPartitions()

2

# Konwersja metadanych na dane 

Jeśli liczba partycji jest już w porządku, to czas na kluczowy moment. <br>
Chcemy, aby każdy z elementów naszego RDD zamienił się z nazwy pliku, na szereg elementów odnoszących się do poszczególnych linii zawartych w tym pliku. 

Potrzebujemy zatem funkcji, która:
* odczyta plik o podanej nazwie 
* dokona ekstracji jego zawartości
* utworzy listę zawierającą poszczególne linie

Funkcję tą wykorzystamy następnie w metodzie `flatMap` na naszym `RDD`. <br>
Reszta będzie *easy peasy*. 

**Uwaga!** <br>
Plik nie będzie znajdował się w lokalnym systemie plików węzła roboczego... będzie znajdował się w systemie plików HDFS!

Aby sobie z tym poradzić, sprawdźmy czy mamy dostępną jeszcze jedną bibliotekę.

In [ ]:
%%sh
pip freeze | grep pydoop

In [18]:
def pdf2txt(fileName):
    
    import PyPDF2

    # Utwórz obiekt odnoszący się do przykładowego pliku
    pdfFileObj = open(fileName, "rb") 
    
    # Utwórz obiekt PdfFileReader 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
    
    lines = []
    
    for page in range(len(pdfReader.pages)): 
        pageObj = pdfReader.pages[page] 
        content = pageObj.extract_text() 
        lines.extend(content.splitlines())
    pdfFileObj.close()
    
    return lines

Sprawdźmy ją. Tym razem będzie to odczyt z systemu plików HDFS.

In [19]:
lines_3gab = pdf2txt("cano-pdf/3gab.pdf")

In [20]:
lines_3gab[:3]

['The Adventure of the Three Gables',
 'Arthur Conan Doyle',
 'This text is provided to you “as-is” without any warranty. No warranties of any kind, expressed or implied, are made to you as to the']

Pozostało nam z niej skorzystać.

In [25]:
lines = fileNames.flatMap(lambda fn: pdf2txt(fn))

Próba generalna

In [28]:
lines.take(2)

['The Adventure of the Three Gables', 'Arthur Conan Doyle']

# Zadania 

Teraz już z górki. Reszta należy do Ciebie. 

**Uwaga!** Na wynikowym RDD, który powinien zawierać dla każdego słowa liczbę jego wystąpień, będziemy wykonywali wiele operacji. <br>
Zadbaj o to, aby każdorazowe użycie tego wynikowego RDD nie powodowało odczytywania plików PDF.

## Zadanie 1

Utwórz obiekt RDD `wordCounts`, który dla każdego słowa liczbę jego wystąpień.

In [36]:
import re
words = lines.flatMap(lambda line: re.split(r'\W+', line))
words.take(2)

['The', 'Adventure']

In [39]:
wordCounts = words.groupBy(lambda word: word).map(lambda pair: (pair[0], len(pair[1])))
wordCounts.take(2)
wordCounts.persist()

PythonRDD[32] at RDD at PythonRDD.scala:53

## Zadanie 2

Znajdź 10 najczęściej wykorzystywanych słów.

In [48]:
wordCounts.sortBy(lambda pair: pair[1], False).take(10)

[('', 42232),
 ('the', 33333),
 ('I', 17288),
 ('and', 16818),
 ('of', 16688),
 ('to', 16038),
 ('a', 15083),
 ('that', 10693),
 ('in', 10359),
 ('was', 9763)]

## Zadanie 3

Znajdź 10 najczęściej wykorzystywanych słów, które składają się z co najmniej 5 liter. 

In [50]:
wordCounts.filter(lambda pair: len(pair[0]) >= 5).sortBy(lambda pair: pair[1], False).take(10)

[('which', 4243),
 ('Holmes', 3044),
 ('there', 2186),
 ('would', 2150),
 ('could', 1843),
 ('should', 1229),
 ('There', 1200),
 ('about', 1096),
 ('before', 1016),
 ('little', 1009)]

## Zadanie 4

Ile razy pojawiło się słowo "Watson"?

In [45]:
wordCounts.filter(lambda pair: pair[0] == "Watson").collect()

[('Watson', 984)]

## Zadanie 5

A ile razy pojawiło się słowo "Moriarty"?

In [46]:
wordCounts.filter(lambda pair: pair[0] == "Moriarty").collect()

[('Moriarty', 49)]